
# Earthquake Exposure Analysis
**Student:** Surya Jamuna Rani Subramanian & Govindharajulu Ramachandran

In this assignment, we analyze the seismic risk for major populated cities using real-time USGS data.
We fetch live data, process it (projecting to meters), index it with a KD-Tree, and calculate exposure metrics.


In [ ]:
# Standard library imports
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# My custom modules
# (I put them in src/earthquake_exposure)
from earthquake_exposure.acquire import get_earthquake_data, get_cities_data
from earthquake_exposure.preprocess import cleanup_gdf, fix_crs_to_metric
from earthquake_exposure.spatial_index import EarthquakeIndex
from earthquake_exposure.metrics import calculate_exposure, apply_normalization, get_final_score
from earthquake_exposure.viz import generate_interactive_dashboard, generate_plotly_map

# This helps to reload my code if I look at it in VS Code
%load_ext autoreload
%autoreload 2

print("Modules loaded!")


## 1. Data Acquisition
We get the earthquake data for the last 90 days (Magnitude 5.0+).
We also load the city data.


In [ ]:
# Fetching data...
print("Downloading earthquakes...")
eq_gdf = get_earthquake_data(days_back=90, min_mag=5.0)
eq_gdf = cleanup_gdf(eq_gdf)

print("Loading cities...")
cities_gdf = get_cities_data()

print(f"Downloaded {len(eq_gdf)} earthquakes and {len(cities_gdf)} cities.")


## 2. Spatial Analysis
We project everything to a Metric CRS (EPSG:4087) so we can measure meters accurately.
Then we build the KD-Tree index.


In [ ]:
# Projecting to meters
eq_m = fix_crs_to_metric(eq_gdf)
cities_m = fix_crs_to_metric(cities_gdf)

# Building the Index
print("Building KDTree...")
index = EarthquakeIndex(eq_m)

# Calculating Risk (Radius = 100km)
print("Calculating exposure...")
full_results = calculate_exposure(cities_m, eq_m, index, radius_km=100)

# Normalize scores (0 to 1)
norm_df = apply_normalization(full_results)
full_results['exposure_score'] = get_final_score(norm_df)

# Filter for interesting results
results_df = full_results[full_results['exposure_score'] > 0.05].copy()
filtered_cities = cities_gdf[cities_gdf['NAME'].isin(results_df['city_name'])]

print("Top 5 Riskiest Cities:")
print(results_df.sort_values('exposure_score', ascending=False).head(5)[['city_name', 'country', 'exposure_score']])


## 3. Visualization
Here we show the relationship between depth and magnitude, and the map of risk.


In [ ]:
# Interactive Plots
fig1, fig2 = generate_interactive_dashboard(eq_gdf, results_df)
fig1.show()
fig2.show()

# The Main Map (Dark Mode)
print("Generating Map...")
fig_map = generate_plotly_map(filtered_cities, eq_gdf, results_df)
fig_map.show()